In [365]:
import pandas as pd               
import numpy as np
import math
from timeit import default_timer as timer
import requests
from astroquery.jplhorizons import Horizons
import datetime
import ephem
import time as tym

In [272]:
import spiceypy as sp
import astropy.coordinates
import re
import sgp4.api as sg
import astropy.units as u
from astropy.coordinates import SkyCoord

In [273]:
import matplotlib.pyplot as plt
import os
import sys
from timeit import default_timer as timer
from astropy.time import Time
from astropy import units as u
from astropy.time import TimeDelta
import datetime as dt
import timeit
import skyfield
# from skyfield.framelib import ecliptic_frame # For rotation matrices
from skyfield.api import EarthSatellite # For time calculations
from skyfield.api import load
from skyfield.api import N,S,E,W, wgs84
from skyfield.positionlib import Barycentric

In [274]:
import warnings
warnings.filterwarnings("ignore")

In [275]:
from numba import jit
e_shadow_angle = np.arctan(690000/150000000)*180/np.pi

In [381]:
#First 10000 LSST obersvations for Main Belt Asteroids (S1)
path = "/data/survey_simulator/baseline_v3.0/by-visit/S0"
dir_list = os.listdir(path)
dflist=[]
dflist.append(pd.read_hdf(path+'/visit-0000000'))
dflist.append(pd.read_hdf(path+'/visit-0010000'))
#dflist.append(pd.read_hdf(path+'/visit-0060000'))
#dflist.append(pd.read_hdf(path+'/visit-0070000'))
dfin=pd.concat(dflist)
dfin.sort_values(['FieldID'], inplace=True)
dfin.groupby(['FieldID']).count()['ObjID']

FileNotFoundError: [Errno 2] No such file or directory: '/data/survey_simulator/baseline_v3.0/by-visit/S0'

In [351]:
names=dfin['ObjID'].values
nNeos=0
for n in names:
    if(n[0:2]=='S0'):
        nNeos=nNeos+1
nNeos

1115745

In [352]:
secperday = 86400
rearthkm = 6370
mu = 398600 
ts = load.timescale()
eph = load('de430t.bsp')

In [353]:
latitude = -30.244633
longitude = -70.74942
elevation = 2647
# Start Time, Stop Time and time step of ephemeris output in Julian Date (UT1)
tme = Time('2023-05-21T14:53:0.0',scale='utc')
dt=24/24
#dt=1/86400
jdstart = tme.jd-dt
jdstop = jdstart+dt
jdstep = 1/86400*60*2
#jdstep=1/86400

In [354]:
def icrf2radec(pos, deg=True):
    """Convert ICRF xyz to Right Ascension and Declination.
    Geometric states on unit sphere, no light travel time/aberration correction.
    
    Parameters:
    -----------
    pos ... real, dim=[n, 3], 3D vector of unit length (ICRF)
    deg ... True: angles in degrees, False: angles in radians
    Returns:
    --------
    ra ... Right Ascension [deg]
    dec ... Declination [deg]
    """
    norm=np.linalg.norm
    array=np.array
    arctan2=np.arctan2
    arcsin=np.arcsin
    rad2deg=np.rad2deg
    modulo=np.mod
    pix2=2.*np.pi    
    if(pos.ndim>1):
        r=norm(pos,axis=1)
        xu=pos[:,0]/r
        yu=pos[:,1]/r
        zu=pos[:,2]/r
    else:
        r=norm(pos)
        xu=pos[0]/r
        yu=pos[1]/r
        zu=pos[2]/r   
    phi=arctan2(yu,xu)
    delta=arcsin(zu)    
    if(deg):
        ra = modulo(rad2deg(phi)+360,360)
        dec = rad2deg(delta)
    else:
        ra = modulo(phi+pix2,pix2)
        dec = delta    
    return ra, dec

In [355]:
def radec2icrf(ra, dec, deg=True):
    """Convert Right Ascension and Declination to ICRF xyz unit vector.
    Geometric states on unit sphere, no light travel time/aberration correction.
    
    Parameters:
    -----------
    ra ... Right Ascension [deg]
    dec ... Declination [deg]
    deg ... True: angles in degrees, False: angles in radians
    Returns:
    --------
    x,y,z ... 3D vector of unit length (ICRF)
    """
    deg2rad=np.deg2rad
    array=np.array
    cos=np.cos
    sin=np.sin
    if(deg):
        a = deg2rad(ra)
        d = deg2rad(dec)
    else:
        a = array(ra)
        d = array(dec)   
    cosd = cos(d)
    x = cosd*cos(a)
    y = cosd*sin(a)
    z = sin(d)    
    return array([x, y, z])

In [356]:
#Observer States
def getObserverStates(observationTimes,observerLatitudeDeg=0,observerLatitudeDirection='N',
                     observerLongitudeDeg=0,observerLongitudeDirection='W',observerElevationMeters=0):
    """Produce observer state vectors at observation times.
    Parameters
    ----------
    observationTimes
        Astropy Time array
    observerLatitudeDeg
        observer WGS84 latitude in degrees (float)
    observerLatitudeDirection
        observer latitude direciton ('N' for North or 'S' for South)
    observerLongitudeDeg
        observer WGS84 longitude in degrees (float)
    observerLatitudeDirection
        observer longitude direciton ('E' for East or 'W' for West)
    observerElevationMeters
        observer elevation in meters
    Returns
    -------
    observer positions
        WGS84 x,y,z observer positions at observation epochs in kilometers.
    observer velocities
        WGS84 dx/dt, dy/dt, dz/dt observer velocity at observation epochs in kilometers per second.
    """
    # load time scale object from skyfield
    ts = load.timescale()
    times = ts.from_astropy(observationTimes)
    if(observerLatitudeDirection == 'S'):
        latDir = S
    elif (observerLatitudeDirection == 'N'):
        latDir = N
    else:
        Error("Observer Latitude Direction unknown. Must be 'N' or 'S'.")
    if(observerLongitudeDirection == 'W'):
        lonDir = W
    elif (observerLongitudeDirection == 'E'):
        lonDir = E
    else:
        Error("Observer Longitude Direction unknown. Must be 'E' or 'W'.")
    observer = wgs84.latlon(observerLatitudeDeg * latDir, observerLongitudeDeg * lonDir, elevation_m=observerElevationMeters)
    observerAt = observer.at(times)
    return np.hstack([observerAt.position.km.T])

In [357]:
def get_ephemeris_by_name(latitude, longitude, elevation, julian_date):
    '''
    Returns the Right Ascension and Declination relative to the observer's coordinates
    for the given satellite's Two Line Element Data Set at inputted Julian Date.

    **Please note, for the most accurate results, an inputted Julian Date close to the TLE epoch is necessary.

    Parameters
    ---------
    name: 'str'
        CelesTrak name of object
    latitude: 'float'
        The observers latitude coordinate (positive value represents north, negative value represents south)
    longitude: 'float'
        The observers longitude coordinate (positive value represents east, negative value represents west)
    elevation: 'float'
        Elevation in meters
    julian_date: 'float'
        UT1 Universal Time Julian Date. An input of 0 will use the TLE epoch.
    tleapi: 'str'
        base API for query

    Returns
    -------
    Name: 'str'
        The name of the query object
    JulianDate: 'float' or list of 'float'
        UT1 Universal Time Julian Date. 
    Right Ascension: 'float'
        The right ascension of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,360)
    Declination: 'float'
        The declination of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [-90,90]
    Altitude: 'float'
        The altitude of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,90]
    Azimuth: 'float'
        The azimuth of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,360)
    Range: 'float'
        Range to object in km
    '''
    name = 'Starlink'
    tleLine1='1 08000U 20001A   23274.00000000  .00000000  00000-0  50000-4 0    01'
    tleLine2='2 08000  53.0000   0.0000 0001000   0.0000 278.6242 15.76641824    04'

    #Cast the latitude, longitude, and jd to floats (request parses as a string)
    lat = float(latitude)
    lon = float(longitude)
    ele = float(elevation)
    
    # Converting string to list
    jul = str(julian_date).replace("%20", ' ').strip('][').split(', ')
   
    # Converting list elements to float
    jd = [float(i) for i in jul]
   
    if(len(jd)>1000):
        raise InvalidAPIUsage("Too many entries requested!",status_code=402)   

    resultList = []
    for d in jd:
        #Right ascension RA (deg), Declination Dec (deg), dRA/dt*cos(Dec) (deg/day), dDec/dt (deg/day),
        # Altitude (deg), Azimuth (deg), dAlt/dt (deg/day), dAz/dt (deg/day), distance (km), range rate (km/s), phaseangle(deg), illuminated (T/F)   
        [ra, dec, dracosdec, ddec, alt, az, 
         #dalt, daz, 
         r, dr, phaseangle, illuminated] = propagateSatellite(tleLine1,tleLine2,lat,lon,ele,d)
        
        resultList.append([d, ra._degrees, dec.degrees, dracosdec, ddec,
                                     alt.degrees, az.degrees, 
                                     #dalt*secperday, daz*secperday, 
                                     r.km, dr, phaseangle, illuminated]) 
    return resultList

In [358]:
def propagateSatellite(tleLine1, tleLine2, lat, lon, elevation, jd, dtsec=1):
    """Use Skyfield (https://rhodesmill.org/skyfield/earth-satellites.html) 
     to propagate satellite and observer states.
     
     Parameters
    ---------
    tleLine1: 'str'
        TLE line 1
    tleLine2: 'str'
         TLE line 2
    lat: 'float'
        The observer WGS84 latitude in degrees
    lon: 'float'
        The observers WGS84 longitude in degrees (positive value represents east, negatie value represents west)
    elevation: 'float'
        The observer elevation above WGS84 ellipsoid in meters
    julian_date: 'float'
        UT1 Universal Time Julian Date. An input of 0 will use the TLE epoch.
    tleapi: 'str'
        base API for query

    Returns
    -------
    Right Ascension: 'float'
        The right ascension of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,360)
    Declination: 'float'
        The declination of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [-90,90]
    Altitude: 'float'
        The altitude of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,90]
    Azimuth: 'float'
        The azimuth of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,360)
    distance: 'float'
        Range from observer to object in km
    """
    
    ts = load.timescale()
    satellite = EarthSatellite(tleLine1,tleLine2,ts = ts)

    #Get current position and find topocentric ra and dec
    currPos = wgs84.latlon(lat, lon, elevation)
    # Set time to satellite epoch if input jd is 0, otherwise time is inputted jd
    if jd == 0: t = ts.ut1_jd(satellite.model.jdsatepoch)
    else: t = ts.ut1_jd(jd)

    difference = satellite - currPos
    topocentric = difference.at(t)
    topocentricn = topocentric.position.km/np.linalg.norm(topocentric.position.km)
    
    ra, dec, distance = topocentric.radec()
    alt, az, distance = topocentric.altaz()
   
    dtday=dtsec/68400
    tplusdt = ts.ut1_jd(jd+dtday)
    tminusdt = ts.ut1_jd(jd-dtday)
    
    dtx2 = 2*dtsec 
    sat = satellite.at(t).position.km

    satn = sat/np.linalg.norm(sat)
    satpdt = satellite.at(tplusdt).position.km
    satmdt = satellite.at(tminusdt).position.km
    vsat = (satpdt - satmdt)/dtx2

    sattop = difference.at(t).position.km
    sattopr = np.linalg.norm(sattop)
    sattopn = sattop/sattopr
    sattoppdt = difference.at(tplusdt).position.km
    sattopmdt = difference.at(tminusdt).position.km
    
    ratoppdt,dectoppdt = icrf2radec(sattoppdt)
    ratopmdt,dectopmdt = icrf2radec(sattopmdt)
    
    vsattop = (sattoppdt - sattopmdt)/dtx2
    
    ddistance = np.dot(vsattop,sattopn)
   
    dra = (ratoppdt - ratopmdt)/dtx2
    ddec = (dectoppdt - dectopmdt)/dtx2
    dracosdec = dra*np.cos(dec.radians)
   
    earth = eph['Earth']
    sun = eph['Sun']
    #print(t,ts.ut1(jd))
    earthp = earth.at(ts.ut1_jd(jd)).position.km
    sunp = sun.at(ts.ut1_jd(jd)).position.km
    earthsun = sunp - earthp
    earthsunn = earthsun/np.linalg.norm(earthsun)
    satsun =  sat - earthsun
    satsunn = satsun/np.linalg.norm(satsun)
    phase_angle = np.rad2deg(np.arccos(np.dot(satsunn,topocentricn)))
    
    #Is the satellite in Earth's Shadow?
    r_parallel = np.dot(sat,earthsunn)*earthsunn
    r_tangential = sat-r_parallel
    
 #   if alt.degrees>0:
  #      print('Satellite is above the horizon')
   # else:
    #    print('Satellite is not above the horizon')    

    illuminated = True
    
    if(np.linalg.norm(r_parallel)<0): 
        if(np.linalg.norm(r_tangential)<rearthkm):
            #print(np.linalg.norm(r_tangential),np.linalg.norm(r))
            #yes the satellite is in Earth's shadow, no need to continue (except for the moon of course)
            illuminated = False
    
    return (ra, dec, dracosdec, ddec, alt, az, 
            distance, ddistance, phase_angle, illuminated)


def my_arange(a, b, dr, decimals=11):
    """
    Better arange function that compensates for round-off errors.
    
    Parameters:
    -----------
    a: 'float'
        first element in range 
    b: 'float'
        last element in range
    dr: 'float'
        range increment
    decimals: 'integer'
        post comma digits to be rounded to
        
    Returns:
    --------
    res: 'numpy array of floats'
        array of numbers between a and b with dr increments
    """
    
    res = [a]
    k = 1
    while res[-1] < b:
        tmp = np.round(a + k*dr,decimals)
        if tmp > b:
            break   
        res.append(tmp)
        k+=1

    return np.asarray(res) 

def tle2ICRFstate(tleLine1,tleLine2,jd):

    #This is the skyfield implementation
    ts = load.timescale()
    satellite = EarthSatellite(tleLine1,tleLine2,ts = ts)

    # Set time to satellite epoch if input jd is 0, otherwise time is inputted jd
    if jd == 0: t = ts.ut1_jd(satellite.model.jdsatepoch)
    else: t = ts.ut1_jd(jd)

    r =  satellite.at(t).position.km
    # print(satellite.at(t))
    v = satellite.at(t).velocity.km_per_s
    return np.concatenate(np.array([r,v]))

def jsonOutput(name,time,ra,dec,dracosdec,ddec, 
               alt, az, 
               #dalt, daz, 
               r, dr, phaseangle, illuminated, 
               precisionAngles=11,precisionDate=12,precisionRange=12):
    """
    Convert API output to JSON format
    
    Parameters:
    -----------
    name: 'str'
        Name of the target satellite
    time: 'float'
        Julian Date
    ra: Skyfield object / 'float'
        Right Ascension 
    dec: Skyfield object / 'float'
        Declination
    alt: Skyfield object / 'float'
        Altitude
    az: Skyfield object / 'float'
        Azimuth
    r: Skyfield object / 'float'
        Range to target
    precisionAngles: 'integer'
        number of digits for angles to be rounded to (default: micro arcsec)
    precisionDate: 'integer'
        number of digits for Julian Date to be rounded to (default: micro sec)
    precisionRange: 'integer'
        number of digits for angles to be rounded to (default: nano meters)   
        
    Returns:
    --------
    output: 'dictionary'
        JSON dictionary of the above quantities
    
    """
    
    #looking up the numpy round function once instead of multiple times makes things a little faster
    myRound = np.round
    output= {"NAME": name,
            "JULIAN_DATE": myRound(time,precisionDate),
            "RIGHT_ASCENSION-DEG": myRound(ra._degrees,precisionAngles),
            "DECLINATION-DEG": myRound(dec.degrees,precisionAngles),
            "DRA_COSDEC-DEG_PER_SEC":  myRound(dracosdec,precisionAngles),
            "DDEC-DEG_PER_SEC": myRound(ddec,precisionAngles),
            "ALTITUDE-DEG": myRound(alt.degrees,precisionAngles),
            "AZIMUTH-DEG": myRound(az.degrees,precisionAngles),
            # "DALT-DEG_PER_SEC": myRound(dalt,precisionAngles),
            # "DAZ-DEG_PER_SEC": myRound(daz, precisionAngles),
            "RANGE-KM": myRound(r.km,precisionRange),
            "RANGE_RATE-KM_PER_SEC": myRound(dr,precisionRange),
            "PHASE_ANGLE-DEG": myRound(phaseangle, precisionAngles),
            "ILLUMINATED": illuminated
            } 
    
    return output   


def icrf2radec(pos, deg=True):
    """
    Convert ICRF xyz to Right Ascension and Declination.
    Geometric states on unit sphere, no light travel time/aberration correction.
    
    Parameters:
    -----------
    pos ... real, dim=[n, 3], 3D vector of unit length (ICRF)
    deg ... True: angles in degrees, False: angles in radians
    Returns:
    --------
    ra ... Right Ascension [deg]
    dec ... Declination [deg]
    """
    norm=np.linalg.norm
    array=np.array
    arctan2=np.arctan2
    arcsin=np.arcsin
    rad2deg=np.rad2deg
    modulo=np.mod
    pix2=2.*np.pi
    
    if(pos.ndim>1):
        r=norm(pos,axis=1)
        xu=pos[:,0]/r
        yu=pos[:,1]/r
        zu=pos[:,2]/r
    else:
        r=norm(pos)
        xu=pos[0]/r
        yu=pos[1]/r
        zu=pos[2]/r
    
    phi=arctan2(yu,xu)
    delta=arcsin(zu)
    
    if(deg):
        ra = modulo(rad2deg(phi)+360,360)
        dec = rad2deg(delta)
    else:
        ra = modulo(phi+pix2,pix2)
        dec = delta
    
    return ra, dec


def uicrf2radec(pos, deg=True):
    """
    Convert ICRF xyz unit vector to Right Ascension and Declination.
    Geometric states on unit sphere, no light travel time/aberration correction.
    
    Parameters:
    -----------
    pos ... real, dim=[n, 3], 3D vector of unit length (ICRF)
    deg ... True: angles in degrees, False: angles in radians
    Returns:
    --------
    ra ... Right Ascension [deg]
    dec ... Declination [deg]
    """
    norm=np.linalg.norm
    array=np.array
    arctan2=np.arctan2
    arcsin=np.arcsin
    rad2deg=np.rad2deg
    modulo=np.mod
    pix2=2.*np.pi
    
    if(pos.ndim>1):
        xu=pos[:,0]
        yu=pos[:,1]
        zu=pos[:,2]
    else:
        xu=pos[0]
        yu=pos[1]
        zu=pos[2]
    
    phi=arctan2(yu,xu)
    delta=arcsin(zu)
    
    if(deg):
        ra = modulo(rad2deg(phi)+360,360)
        dec = rad2deg(delta)
    else:
        ra = modulo(phi+pix2,pix2)
        dec = delta
    
    return ra, dec

In [359]:
@jit
def crossTrackDistanceVectors(P1,P2,P3):
    # check if the point is within the line connecting P1 and P2
    # returns distance in distance units
    v1=P3-P1
    v2=P3-P2    
    if(np.dot(v1,v2)<0): 
        S = np.cross(P1,P2)
        dot = np.dot(S,P3)
        normS = np.linalg.norm(S)
#     print('S',S)
#     print('dot',dot)
        dist = np.arctan2(dot/normS, np.sqrt(1 - (dot/normS)**2 )) 
    else:
        dist=1e10
    return dist

In [360]:
field_ids = dfin['FieldID'].unique()
dates = dfin['FieldMJD'].unique()
print(dates)
print(len(field_ids))

[60218.00180556 60218.0022596  60218.00271525 ... 60246.0715398
 60246.07202907 60246.07248742]
20000


In [361]:
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
with open('sats.txt') as f:
    starlinks = f.read().splitlines() 
# split up the list into 3-line lists for each satellite
chunks = [starlinks[n:n + 3] for n in range(0, len(starlinks), 3)]
satdf=pd.DataFrame(chunks,columns=["SatID",'TLELine1',"TLELine2"])
org1 = str(satdf['TLELine1'])
tleLine1 = org1[5:]
org2 = str(satdf['TLELine2'])
tleLine2 = org2[5:]

satdf['SatNumber']=satdf.index
print("There are " + str(len(chunks)) + " satellites")

There are 5 satellites


In [362]:
time_f = 2460230.798655486
time = Time(time_f, format='jd', scale='utc')

ab=get_ephemeris_by_name(latitude, longitude, elevation, time_f)
print(ab)
#output for ab is [jd,ra,da,dracosdec,ddec,altitude,azimuth,range,rangerate]
jd = [row[0+1] for row in ab]
ra = [row[1+1] for row in ab]
da = [row[2+1] for row in ab]
dracosdec = [row[3+1] for row in ab]  
ddec = [row[4+1] for row in ab]
azimuth = [row[6+1] for row in ab]   
altitude = [row[5+1] for row in ab]
ranges = [row[7+1] for row in ab]   
rangerate = [row[8+1] for row in ab]

[[2460230.798655486, 238.86897936737392, -10.352184045953594, 0.053032274597549306, 0.017922708813232546, -49.32554397786322, 181.2560681590998, 10142.84668865267, 2.364644293266202, 139.43783563823308, True]]


In [380]:
#start = timer()
FOV_dist_lim = 5/180*np.pi #3 degrees distance to FOV for a satellite to be excluded
rearth = 6371
LSSTExpDays = 30/86400
distlim = 3/180/3600*np.pi # 3 arcsecond distance from satellite track
shadow=0
affected=[]

start_time = tym.time()
# Check each satellite against each FIELD
for f in field_ids: #[100:1000]:
    data = dfin[dfin['FieldID']==f]  # all objects in the field
    sun_vector = data[['Obs-Sun(J2000x)(km)', 'Obs-Sun(J2000y)(km)', 'Obs-Sun(J2000z)(km)']][0:1].values[0]
    sv = sun_vector/np.linalg.norm(sun_vector)
    ra_f = data['AstRA(deg)']  # RA for all objects
    dec_f = data['AstDec(deg)']  # Dec for all objects
    range_f = data['AstRange(km)'] # Distance for all objects
    time_f = data['FieldMJD'].median()
    objID_f = data['ObjID']
    xyzu = radec2icrf(ra_f, dec_f, deg=True)
    FOV_mean_direction=np.median(xyzu,axis=1)
    time = Time(time_f, format='jd', scale='utc') + 2400000.5
    #observer_state = getObserverStates(time,observerLatitudeDeg=30.24506,observerLatitudeDirection='S',
    #              observerLongitudeDeg=70.74913,observerLongitudeDirection='W',observerElevationMeters=2663) 
    
    for index, row in satdf.iterrows():    
        ab=get_ephemeris_by_name(latitude, longitude, elevation, time)
        #output for ab is [jd,ra,da,dracosdec,ddec,altitude,azimuth,range,rangerate]
        jd = [row[0+1] for row in ab]
        ra = [row[1+1] for row in ab]
        da = [row[2+1] for row in ab]
        dracosdec = [row[3+1] for row in ab]  
        ddec = [row[4+1] for row in ab]
        azimuth = [row[6+1] for row in ab]   
        altitude = [row[5+1] for row in ab]
        ranges = [row[7+1] for row in ab]   
        rangerate = [row[8+1] for row in ab]
        
        sat_r = radec2icrf(ra, da, deg=True)
        sat_r = np.transpose(sat_r)
        sat_r = np.array(sat_r)
        sat_r = sat_r.ravel()
        # print('time',time,sat_r)
        #Is the satellite in Earth's Shadow?
        r_parallel = np.dot(sat_r,sv)*sv
        r_tangential = sat_r-r_parallel 

        #if(np.linalg.norm(r_tangential)<rearth):
            #print(np.linalg.norm(r_tangential),np.linalg.norm(r))
            #yes the satellite is in Earth's shadow, no need to continue (except for the moon of course)
         #   shadow=shadow+1
         #   continue
#         else:
#             print('not in shadow')
            # break   
        #Is the satellite near the FOV?
        sat_d = np.linalg.norm(sat_r)
        #correct this!! obs = observer_state[0:3]
        nvar = sat_r-obs
        obs_norm = obs/obs_mag
        ang_dist = np.arccos(np.dot(FOV_mean_direction,nvar)/(np.linalg.norm(nvar)))

        if(ang_dist > FOV_dist_lim):
            continue
        
        tminus = Time(time.mjd-LSSTExpDays/2,format='mjd',scale='utc')
        tplus = Time(time.mjd+LSSTExpDays/2,format='mjd',scale='utc')
#         print((tminus-tplus)*86400,'timestuff')
#         print(tminus.jd)
        # do a more thorrow analysis by propagating the satellite to +15 and -15 secs and check for cross track distance
        R1,V1 = getSatelliteStates(tminus,row['TLELine1'],row['TLELine2'])
        R2,V2 = getSatelliteStates(tplus,row['TLELine1'],row['TLELine2'])
#         print('R1, R2',np.array(R1),np.array(R2))
#         print('R1-R2',np.array(R1)-np.array(R2))
        
        OBSERV1 = getObserverStates(tminus,observerLatitudeDeg=30.24506,observerLatitudeDirection='S',
                  observerLongitudeDeg=70.74913,observerLongitudeDirection='E',observerElevationMeters=2663)
        OBSERV2 = getObserverStates(tplus,observerLatitudeDeg=30.24506,observerLatitudeDirection='S',
                  observerLongitudeDeg=70.74913,observerLongitudeDirection='E',observerElevationMeters=2663)
   
        O1 = np.array(R1)-OBSERV1
        O2 = np.array(R2)-OBSERV2
        
        P1 = O1/np.linalg.norm(O1)
        P2 = O2/np.linalg.norm(O2)
        
        i = 0
        mincdist = 1
        minsat = ""
        
        for obj in objID_f: 
            P3 = xyzu[:,i]
            # print('P1',P1,'P2',P2,'P3',P3)
            cdist = np.abs(crossTrackDistanceVectors(P1,P2,P3))
            #print('cdist [arcsec]',cdist*3600*180/np.pi)
            # print('cdist',cdist)
            adist = np.arccos(np.dot(P1,P3))
            if (cdist < mincdist):
                    mincdist = cdist
                    minsat = row.SatID
            if ((cdist <= distlim) & (adist <= FOV_dist_lim)):
                affected.append([time,cdist*3600*180/np.pi,obj,f,ra_f.values[i],dec_f.values[i],row,P1,P2,P3])
                print([time,cdist*3600*180/np.pi,obj,f,ra_f.values[i],dec_f.values[i],row,P1,P2,P3])
            i=i+1
        if (cdist < 1e9):
            print('minimum distance ["]',mincdist*3600*180/np.pi,'of satellite ',minsat)
    #radec2icrf(ra, dec, deg=True)
    #Is it in the extended FOV?
    

end_time = tym.time()

print("time",(end_time - start_time)/60)
print(affected)


2.888355704859829
2.888355704859829
2.888355704859829
2.888355704859829
2.888355704859829
2.8381127066389547
2.8381127066389547
2.8381127066389547
2.8381127066389547
2.8381127066389547
2.8786881948656933
2.8786881948656933
2.8786881948656933
2.8786881948656933
2.8786881948656933
2.348769026617369
2.348769026617369
2.348769026617369
2.348769026617369
2.348769026617369
2.298231428365641
2.298231428365641
2.298231428365641
2.298231428365641
2.298231428365641
2.240273609149027
2.240273609149027
2.240273609149027
2.240273609149027
2.240273609149027
2.265112593769917
2.265112593769917
2.265112593769917
2.265112593769917
2.265112593769917
2.2441462458245605
2.2441462458245605
2.2441462458245605
2.2441462458245605
2.2441462458245605
2.2373319284158573
2.2373319284158573
2.2373319284158573
2.2373319284158573
2.2373319284158573
2.208354998266628
2.208354998266628
2.208354998266628
2.208354998266628
2.208354998266628
2.2122093194988124
2.2122093194988124
2.2122093194988124
2.2122093194988124
2.21

In [382]:
 5/180*np.pi

0.08726646259971647